# Assignment 1


In [ ]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [ ]:
#create co-occurrence matrix
shape = (V, V)
matrix = np.zeros(shape)
np.fill_diagonal(matrix, 0)
for sentence in corpus:
    for i in range(0, len(sentence)):
        coword = sentence[i+1:i+window_size_corpus+1]
        for j in coword:
            matrix[sentence[i], j] = matrix[sentence[i], j] + 1
matrix_trans = matrix.transpose()
co_occurence_matrix = np.add(matrix, matrix_trans)

In [ ]:
#normalize the words by dividing the co-occurence frequency with the total frequency of one word
for i in range(1, V):
    total_tf = np.sum(co_occurence_matrix[:,i])
    if total_tf > 0: #some words do not co-occur with other words, as they are the only words in the sentence (for instance 797)
        for j in range(1,V):
            co_occurence_matrix[j,i] = co_occurence_matrix[j,i]/total_tf

#find cosine similarity to Alice, Dinah and Rabbit
alice_index = tokenizer.word_index['alice']
dinah_index = tokenizer.word_index['dinah']
rabit_index = tokenizer.word_index['rabbit']
#create similarity matrix
sim = cosine_similarity(co_occurence_matrix)
sim_alice_dinah = sim[alice_index, dinah_index]
sim_alice_rabit = sim[alice_index, rabit_index]
sim_dinah_rabit = sim[dinah_index, rabit_index]
print("Cosine similarity Alice - Dinah: %.4f" %sim_alice_dinah)
print("Cosine similarity Alice - Rabbit: %.4f" %sim_alice_rabit)
print("Cosine similarity Dinah - Rabbit: %.4f" %sim_dinah_rabit)

In [ ]:
#find the closest words to Alice
nbrs = nn(n_neighbors=6, algorithm='auto').fit(co_occurence_matrix)
distances, indices = nbrs.kneighbors(co_occurence_matrix)

print(indices[alice_index])
for word, index in tokenizer.word_index.items():    # for name, age in list.items():  (for Python 3.x)
    if index in indices[alice_index]:
        print(word)

In [ ]:
print('The length of the co-occurence matrix is {}'.format(len(co_occurence_matrix)))

Trivially, the closest word to Alice is Alice. However, all other works are not in line with what one would expect. Having read the first lines of Alice in Wonderland, it is for example not clear what the connection is between 'Alice' and 'foot'. A possible explanation is that the dimensionality of the data is high, as the co-occurency of two terms is compared with all (1182-2=)1180. Because of this high dimensionality, the distance difference between the nearest and farthest neighbor is low. To illustrate this, we computed the 100 nearest neighbours to 'Alice' below and compared the longest of these 100 distances with the smallest distance bigger than zero. As can be seen in the output of the code below, the nearest neighbour is only 1.9% closer in terms of distance than the 99th nearest neighbour.

In [ ]:
nbrs_100 = nn(n_neighbors=1000, algorithm='auto').fit(co_occurence_matrix)
distances_1000, indices_1000 = nbrs.kneighbors(co_occurence_matrix)

print(distances[alice_index]/np.average(distances_1000[alice_index]))
print(distances[alice_index]/np.max(distances_1000[alice_index]))
np.max(np.max(distances_1000[alice_index])/distances[alice_index][1:])

In [ ]:
#percentage of cells with zeros
print('The ratio of cells with zeros is {:.4f}'.format
      (np.where(co_occurence_matrix==0)[0].size/co_occurence_matrix.size))

Discussion of the drawback of a term-term co-occurence matrix:
- A co-occurence matrix is very sparse. In this data set, 98,16% of the cells in the word-word co-occurence matrix are zeros. 
- The matrix tends to be very large. For the first 700 lines of the book Alice in Wonderland, we already need a matrix of size 1183x1183. 
- The frequency of occurence can be skewed and non-discriminative. Words like 'the', 'and' and 'to' are not very discriminative, but the impact on the calculation of word similarity of these words is equally big as the impact of more discriminative words. However, this is not necessary a drawback of the use of the word-word co-occurence matrix, as one can adjust for this by using weighting terms.

In [ ]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = [co_occurence_matrix]
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    #Dit werkt wel, maar dan krijg je later weer problemen
#     f.write(" ".join(map(str, list(vectors[0][i,:]))))
    f.write("\n")
f.close()

#reopen your file as follows
co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow|
def gen(corpus, window_size, V):
    maxlen = window_size * 2
    for sentence in corpus:
        L = len(sentence)
        for index, word in enumerate(sentence):
            context = []
            labels = []
            s = index - window_size
            e = index + window_size + 1
            
            #Add context word (cat is hungry) if  index is 1 then context is cat and hungry
            context.append([sentence[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)
            
            x = sequence.pad_sequences(context, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            #return generator.
            yield (x, y)

In [4]:
dim = 50
cbow_model_50 = Sequential()
cbow_model_50.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_50.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_50.add(Dense(V, activation='softmax'))
#Compile
cbow_model_50.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_50.train_on_batch(x, y)
    print(ite, loss)

0 41686.85704231262
1 39117.07442212105
2 38909.004811286926
3 38815.759677529335
4 38761.797519654036
5 38724.254935741425
6 38676.11721082032
7 38627.97741906345
8 38588.04490805417
9 38562.02371109277


In [5]:
dim = 150
cbow_model_150 = Sequential()
cbow_model_150.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_150.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_150.add(Dense(V, activation='softmax'))
#Compile
cbow_model_150.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_150.train_on_batch(x, y)
    print(ite, loss)

0 41624.23430228233
1 38758.4873585701
2 38354.18559038639
3 38128.79920192063
4 37915.916541829705
5 37717.64857311547
6 37544.240869909525
7 37407.07984884456
8 37298.178133709356
9 37203.42670722678


In [ ]:
dim = 300
cbow_model_300 = Sequential()
cbow_model_300.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_300.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_300.add(Dense(V, activation='softmax'))
#Compile
cbow_model_300.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_300.train_on_batch(x, y)
    print(ite, loss)

In [ ]:
#Write results
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))
#Write results
cbow_model.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [6]:
import operator
def anology_m(model, word_one, word_two, word_three):
    keyvector = model.get_weights()[0]
    index_one = tokenizer.word_index[word_one]
    index_two = tokenizer.word_index[word_two]
    index_three = tokenizer.word_index[word_three]
    result = keyvector[index_two] - keyvector[index_one] + keyvector[index_three]
    
    closest_words = {}
    closest_word = ""
    min_score = -1
    for word, index in tokenizer.word_index.items():
        if(index not in [index_one, index_two, index_three]):
            sim_score = cosine_similarity([result, keyvector[index]])[0][1]
            closest_words[word] = sim_score
            if(sim_score > min_score):
                min_score = sim_score
                closest_word = word
    sorted_x = sorted(closest_words.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_x[:10]

In [17]:
#Happy not in dataset hence we catch keyerror
def test_acc(model):
    hit_1 = 0
    hit_5 = 0
    hit_10 = 0
    file = open('analogy_alice.txt')
    lines = file.readlines()
    for line in lines:
        try:
            tokens = line.strip().split(' ')
            result = anology_m(model, tokens[0], tokens[1], tokens[2])
            if(tokens[3] == result[0]):
                hit_1 += 1
            elif(tokens[3] in result[:5]):
                hit_5 += 1
            elif(tokens[3] in result):
                hit_10 += 1
        except KeyError:
            pass
    return hit_1, hit_5, hit_10

In [16]:
res_cbow_50_1, res_cbow_50_2, res_cbow_50_3 = test_acc(cbow_model_50)
print(res_cbow_50_1)
print(res_cbow_50_2)
print(res_cbow_50_3)

[('among', 0.81847054), ('once', 0.81836534), ('far', 0.8182786), ('trotting', 0.8157228), ('several', 0.8156117), ('catching', 0.807878), ('ma', 0.8023381), ('how', 0.80214024), ('half', 0.8021327), ('their', 0.7994348)]
[('his', 0.8714411), ('william', 0.85736096), ('too', 0.8495076), ('kid', 0.846555), ('their', 0.84449244), ('times', 0.84395695), ('this', 0.8395642), ('very', 0.8388386), ('an', 0.8369546), ('its', 0.8328292)]
[('ready', 0.5587122), ('bathing', 0.5165816), ('severely', 0.467151), ('there', 0.45625997), ('it', 0.45406643), ('surprised', 0.45232335), ('went', 0.44879824), ('couldn', 0.44581386), ('printed', 0.44305977), ('just', 0.44251865)]
[('it', 0.84654427), ('alice', 0.7051682), ('idea', 0.61104673), ('anxiously', 0.568317), ('you', 0.56314445), ('went', 0.5527689), ('as', 0.53755677), ('them', 0.519373), ('deeply', 0.50861925), ('glass', 0.50669694)]
[('an', 0.8159269), ('your', 0.8151512), ('times', 0.80560535), ('ever', 0.8039189), ('beg', 0.7923994), ('two', 

[('shook', 0.62245613), ('so', 0.5898092), ('there', 0.5736131), ('again', 0.5617761), ('rate', 0.53204125), ('brave', 0.52037454), ('late', 0.5195184), ('signify', 0.5192528), ('were', 0.51773304), ('here', 0.5136372)]
0
0
0


In [12]:
res_cbow_150_1, res_cbow_150_2, res_cbow_150_3 = test_acc(cbow_model_150)
print(res_cbow_150_1)
print(res_cbow_150_2)
print(res_cbow_150_3)

0
0
0


In [ ]:
res_cbow_300 = test_acc(cbow_model_300)